In [ ]:
import pandas as pd
import numpy as np
import glob
import datetime
import re


csv_file_list= glob.glob('./csv/*.csv')

point_names= {
    '32419938': 'Klessen ca. 1 km südöstl. OP',
    '32409933': 'Rhinow, Sportplatz UP',
    '31419861': 'Segeletz',
    '32419870': 'Michaelisbruch Wald',
    '32419874': 'Michaelisbruch Haus Nr.18 Fl',
    '32419840': 'Wutzetz östlich B5',
    '31391801': 'Babe',
    '32419851': 'Friesack Bahnüberführung',
    '32419819': 'Friesack an der Kirche neu OP',
    '31400830': 'Neustadt Birkenweg',
    '32419879': 'Dreetz 3 km südöstl. OP',
    '32409905': 'Klessen Ziegelei',
    '31401602': 'Sieversdorf Weg n.Goldbeck',
    '31409918': 'Dreetz Waldeck'
}

dateparser= lambda x: datetime.datetime.strptime(str(x), '%d.%m.%Y')
df= pd.DataFrame()

for csv_file in csv_file_list:
    point_number= re.sub(r'^.*\/exp(\d+)_.*$', r'\1', str(csv_file))
    df_tmp= pd.read_csv(csv_file, index_col= 0, parse_dates= [ 0 ], date_parser=dateparser, header= None, skiprows= 2, encoding= 'iso8859_1', sep= ';', decimal= ',')
    df_tmp.rename(columns= { 1: point_number + ' ' + point_names[point_number] }, inplace= True)
    df_tmp.index.rename('Date', inplace= True)
    df= df.join(df_tmp, how= 'outer')

df.index.rename('Date', inplace= True)

# Total number of data rows
num_rows= len(df.index)
print(str(num_rows).rjust(4) + ' sample dates')


print('\n\n')
for column in df:
    print(str(df[column].isna().sum()).rjust(4) + ' missing values for station "' + column + '"')

# change column data types to float64
for column in df:
    df[column]= df[column].astype(np.float64)

# resample some kind of weekly data to a daily series
# replace up to 32 consecutive missing (daily) values
print('\nrunning resample and interpolate: interval= "D", method= "linear"\n\n')
df= df.resample('D').interpolate(method= 'linear', limit= 32)


# insert column with a simple count index for fitting
df.insert(0, 'Range', range(0, len(df)))

print('\n\n')
df.info()
print('\n\nFirst data rows:\n')
df.head(20)


In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats

colors=  [ 'blue', 'green', 'red', 'cyan', 'magenta', 'salmon', 'lawngreen', 'dodgerblue', 'blueviolet', 'grey', 'darkorange', 'darkgoldenrod', 'lime', 'cornflowerblue', 'blueviolet', 'brown', 'gold', 'teal', 'olive' ]
ci= 0;
pi= 0;


print('{:<8}\t{:<30}\t{:<6}\t{:<12}\t{:<12}\t{:<12}\n'.format('Nummer', 'Name', 'Wert', 'D’Agostino K^2', 'Shapiro-Wilk', 'Anderson-Darling'))

fig, ax= plt.subplots(len(df.columns) // 3, 3, figsize= (25, 35))

for column in df:
    if column == 'Range':
        continue
    # drop missing values from dataframe and convert to values array
    col_clean= df[column].dropna().to_numpy()

    # D’Agostino’s K^2 Test, Documentation: https://docs.scipy.org/doc/scipy/reference/index.html
    k2, p_k2= stats.normaltest(col_clean, axis= None)
    alpha = 0.05
    if p_k2 > alpha:
        k2_gauss= 'ja'
    else:
        k2_gauss= 'nein'

    # Shapiro-Wilk Test
    sw, p_sw= stats.shapiro(col_clean)
    if p_sw > alpha:
        sw_gauss= 'ja'
    else:
        sw_gauss= 'nein'

    ad_result= stats.anderson(col_clean)
    ad= ad_result.statistic
    for i in range(len(ad_result.critical_values)):
        if ad_result.significance_level[i] != 1.0:
            continue
        ad_sl, ad_cv = ad_result.significance_level[i], ad_result.critical_values[i]
        if ad_result.statistic < ad_result.critical_values[i]:
            ad_gauss= 'ja'
        else:
            ad_gauss= 'nein'

    print('{:<8}\t{:<30}\t{:<6}\t{:<10}\t{:<10}\t{:<10}'.format(column[0:8], column[9:], 'p', '{:.5E}'.format(p_k2), '{:.5E}'.format(p_sw),  ''))
    print('{:<8}\t{:<30}\t{:<6}\t{:<12}\t{:<12}\t{:<12}'.format('', '', 'stat', '{:.5E}'.format(k2), '{:.5E}'.format(sw), '{:.5E}'.format(ad)))
    print('{:<8}\t{:<30}\t{:<6}\t{:<12}\t{:<12}\t{:<12}'.format('', '', 'Gauss', k2_gauss , sw_gauss, ad_gauss))

    ax[pi // 3][pi % 3].hist(col_clean, color= colors[ci], label= column)
    ax[pi // 3][pi % 3].set_title(column)

    pi= (pi + 1)
    ci= (ci + 1) % len(colors)

print('\nHistogramme\n')
fig.show()
fig.savefig('plots/histograms.png', dpi= 300)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

ci= 0;
pi= 0;

fig, ax= plt.subplots(len(df.columns) - 1, 1, figsize= (25, 55))

print('\nTrends calculated with linear regression:\n')

print('{:<8}\t{:<30}\t{:<16}\n'.format('Nummer', 'Name', 'Trend'))
# run least squares linear regression with two different libraries to check if everything is fine
for column in df:
    if column == 'Range':
        continue
    # drop missing values from dataframe
    col_clean= df[column].dropna()
    # run regression method 1 with numpy polynomial fit with order 1
    coeffs_polyfit= np.polyfit(range(0, len(col_clean)), col_clean, 1)
    # create matrix from values and run regression method 2 with numpy linear leastsquares
    a= np.vstack([range(0, len(col_clean)), np.ones(len(col_clean))]).T
    coeffs_lstsq= np.linalg.lstsq(a, col_clean, None)[0]
    # print to check if results of polyfit and lstsq are equal
    #print(coeffs_polyfit, '\n', coeffs_lstsq)
    # print result table line
    print('{:<8}\t{:<30}\t{:+.6f} cm/a'.format(column[0:8], column[9:], coeffs_polyfit[0] * 36500))

    # create y values to plot trend
    yn = np.polyval(coeffs_polyfit, df.Range)
    # plot values and trend
    ax[pi].plot(df.index, df[column], c= colors[ci], label= column)
    ax[pi].plot(df.index, yn, c= colors[ci])
    ax[pi].legend(handles= [ mpatches.Patch(color= colors[ci], label= column + ':  Trend: ' + '{:.6f}'.format(coeffs_polyfit[0] * 36500) + ' cm/a') ], fontsize=18)
    pi= (pi + 1)
    ci= (ci + 1) % len(colors)

print('\n')
fig.show()
fig.savefig('plots/trends_linear.png', dpi= 300)

In [ ]:
%%capture
!pip install pymannkendall

In [ ]:
import pymannkendall as mk
 
mann_kendall= pd.DataFrame(index= [ 'trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept' ])
mann_kendall_seasonal= pd.DataFrame(index= [ 'trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept' ])

# perform Mann-Kendall test
for column in df:
    if column == 'Range':
        continue
    mann_kendall[column]= mk.original_test(df[column].dropna())
    mann_kendall_seasonal[column]= mk.seasonal_test(df[column].dropna(), period= 365)

In [ ]:
mann_kendall.head(10)

In [ ]:
mann_kendall_seasonal.head(10)

In [ ]:
print('\nTrends calculated with Mann-Kendall Test:\n')

print('{:<8}\t{:<30}\t{:<12}\t{:<22}\n'.format('Nummer', 'Name', 'Mann-Kendall', 'Mann-Kendall seasonal'))
# run least squares linear regression with two different libraries to check if everything is fine
for column in df:
    if column == 'Range':
        continue
    print('{:<8}\t{:<30}\t{:+.6f} cm/a\t{:+2.6f} cm/a'.format(column[0:8], column[9:], mann_kendall[column].slope * 36500, mann_kendall_seasonal[column].slope * 100))


In [ ]:
# create 365d rolling mean and std dev
mean_y_rolling= pd.DataFrame()
std_y_rolling= pd.DataFrame()
for column in df:
    if column == 'Range':
        continue
    mean_y_rolling[column]= df[column].rolling('365D').mean()
    std_y_rolling[column]= df[column].rolling('365D').std()

In [ ]:
import matplotlib.pyplot as plt

ci= 0;
ax= []
axm= []

for column in df:
    if column == 'Range':
        continue
    ax_tmp= df[[column]].plot(xlabel= 'date', ylabel= str(column), figsize= (25, 3), subplots= False, legend= False, c= colors[ci])
    ci= (ci + 1) % len(colors)
    ax.append(ax_tmp)
    ax_tmp.fill_between(mean_y_rolling.index,
                 (mean_y_rolling[column] - std_y_rolling[column]).values,
                 (mean_y_rolling[column] + std_y_rolling[column]).values, alpha=.5, color= 'green')
    axm_tmp= mean_y_rolling[column].plot(ax= ax_tmp, label= 'rolling mean 365d')
    axm_tmp.legend(['groundwater level [m above NHN]', 'std deviation', 'rolling mean 365d'], fontsize=16)
    axm.append(axm_tmp)
    plt.savefig('plots/rolling_mean_' + column + '.png', dpi= 300)

In [ ]:
!pip install statsmodels
import statsmodels.api as sm
from cycler import cycler
from statsmodels.tsa.seasonal import STL


plt.rc('figure', figsize= (20, 10))
ci= 0;

for column in df:
    if column == 'Range':
        continue
    plt.rcParams['axes.prop_cycle']= cycler(color= [ colors[ci] ])
    ci= (ci + 1) % 10
    #seasonal_decomp= sm.tsa.seasonal_decompose(df[[column]].dropna(), model= 'additive', period= 365)
    seasonal_decomp= STL(df[[column]].dropna(), period= 365).fit()
    axd= seasonal_decomp.plot()
    axd.legend([ column ], fontsize=18)
    plt.savefig('plots/seasonal_decompose_' + column + '.png', dpi= 300)